In [16]:
import openai
import csv

# Get API key & configure OpenAI API
with open('.SECRET') as f:
    secret = f.read().strip()
openai.organization = "org-wAc6SximlF8c4j1M4IepuYrV"
openai.api_key = secret

# Load data
with open('text-simplification-new-wikipedia.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)
    data = [row for row in reader]

# Will only use the sentences
data = [str(x[0]) for x in data]

In [46]:
sample_prompt = """
Sentence: The library needs to be configured with your account's secret key.
Simplify the above sentence.
"""

def callGPT(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002", 
        prompt=prompt, 
        max_tokens=256)
    responses = [x['text'] for x in response['choices']]
    if len(responses) > 1:
        print(f"More than one response! {responses}")
    responses = responses[0]
    return responses

callGPT(sample_prompt)

In [69]:
prompts = {
"naive": """
Sentence: %s
Simply the above sentence.
""",
"a_lot": """
Sentence: %s
Simply the above sentence a lot.
""",
"content_a_lot": """
Sentence: %s
Simply the content in the above sentence a lot.
""",
"as_possible": """
Sentence: %s
Simply the above sentence as much as possible.
""",
"asset_instructions": """
You are given a sentence that need to be rewritten so that they use simpler English. This means that you should reduce the number of difficult words or idioms, simplify complex phrasing, delete information that may not be relevant, and make the sentence more straight-forward. This could be accomplished by applying different transformations to the original sentences. In this task, we ask you to use paraphrasing, compression and/or sentence splitting.

Sentence: %s
Simplified sentence: """
}

In [50]:
# Example using the first prompt
sentence = "The proletarians have nothing to lose but their chains."
print(prompts[2][1:].replace('%s', sentence))

Sentence: The proletarians have nothing to lose but their chains.
Simply the content in the above sentence a lot.



In [61]:
# sent = data[10]
# prompt_id = 3
# prompt = prompts[prompt_id][1:].replace('%s', sent)

# Danger! This costs ~$0.3 for 20 sentences
out = []
for prompt in prompts:
    prompt_out = []
    for sent in data:
        prompt_out.append(callGPT(prompt[1:].replace('%s', sent)))
    out.append(prompt_out)

In [73]:
out_new = {}
for i in range(len(out)):
    prompt_out_new = []
    for sent in out[i]:
        if len(sent) > 0 and sent[0] == '\n':
            sent = sent[1:]
        prompt_out_new.append(sent)
    out_new[list(prompts.keys())[i]] = prompt_out_new

In [75]:
import json
with open('gpt3-output.json', 'w') as f:
    json.dump(out_new, f)